# Encoding

In [2]:
cd ..

/home/simonchuth/Desktop/git_repo/patentAI


In [1]:
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import tensorflow as tf

In [3]:
encoder_path = 'encoder/use4_model/'
encoder = hub.load(encoder_path)

In [4]:
import pickle
pkl_path = 'pkl_files/sample_data/ipos_data.pkl'
with open(pkl_path, 'rb') as pkl_file:
    dataset = pickle.load(pkl_file)

In [6]:
import random

In [7]:
random.shuffle(dataset)

In [10]:
test_ratio = 0.1
test_size = int(len(dataset) * test_ratio)
test_set = dataset[:test_size]
train_set = dataset[test_size:]

In [18]:
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.text import text_to_word_sequence

term_pattern = r'".+?"'

In [49]:
end_word = np.zeros((1,512))
end_tensor = tf.convert_to_tensor(end_word)
end_tensor = tf.cast(end_tensor, tf.float32)
end_tensor

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

In [50]:
context_tensor = []
definition_tensor = []
terms = []
for app in tqdm(train_set):
    intro = app[0]
    claims = ' '.join(app[1])
    context = [intro, claims]
    context = encoder(context)
    context = tf.reshape(context, [1024])
    context_tensor.append(context)
    
    definition = app[2]
    term = re.findall(term_pattern, definition)[0]
    terms.append(term)
    term_tensor = encoder([term])
    definition_tokens = text_to_word_sequence(definition)
    definition_tokens = encoder(definition_tokens)
    definition_tokens = tf.concat([definition_tokens, end_tensor], axis = 0)
    definition_tensor.append(definition_tokens)

100%|██████████| 12132/12132 [04:04<00:00, 49.65it/s]


In [52]:
train_pkl = 'pkl_files/sample_data/train_tensor.pkl'

combine = [context_tensor, definition_tensor]
with open(train_pkl, 'wb') as pkl_file:
    pickle.dump(combine, pkl_file)

In [54]:
context_tensor = []
definition_tensor = []
terms = []
for app in tqdm(test_set):
    intro = app[0]
    claims = ' '.join(app[1])
    context = [intro, claims]
    context = encoder(context)
    context = tf.reshape(context, [1024])
    context_tensor.append(context)
    
    definition = app[2]
    term = re.findall(term_pattern, definition)[0]
    terms.append(term)
    term_tensor = encoder([term])
    definition_tokens = text_to_word_sequence(definition)
    definition_tokens = encoder(definition_tokens)
    definition_tokens = tf.concat([definition_tokens, end_tensor], axis = 0)
    definition_tensor.append(definition_tokens)
    
test_pkl = 'pkl_files/sample_data/test_tensor.pkl'

combine = [context_tensor, definition_tensor]
with open(test_pkl, 'wb') as pkl_file:
    pickle.dump(combine, pkl_file)

100%|██████████| 1347/1347 [00:26<00:00, 49.94it/s]


# Model building

In [ ]:
from tensorflow.keras import Sequential